<a href="https://colab.research.google.com/github/fahimku2020/fahimku2020/blob/main/Faiss_clustering_good_wikipedia_semantic_similarity_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install wikipedia
!pip install nltk
!pip install sentence-transformers
!pip instal faiss-cpu
!pip install scikit-learn

ERROR: unknown command "instal" - maybe you meant "install"


In [5]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 60.2 MB/s eta 0:00:00


In [8]:
import wikipedia
import nltk
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from functools import lru_cache

# Download required NLTK resources (if not already present)
nltk.download('punkt_tab')
nltk.download('stopwords')

# Configuration
EMBEDDING_MODEL = "all-mpnet-base-v2"  # Powerful sentence transformer model
NUM_CLUSTERS = 10  # Adjust as needed
CHUNK_SIZE = 500   # Process Wikipedia data in chunks

# Initialize sentence transformer model
model = SentenceTransformer(EMBEDDING_MODEL)


@lru_cache(maxsize=100)  # Cache Wikipedia page content
def get_wikipedia_content(page_title):
    try:
        page = wikipedia.page(page_title, auto_suggest=False)  # Disable auto-suggest for more control
        return page.content
    except wikipedia.exceptions.PageError:
        print(f"Error: Page '{page_title}' not found.")
        return None
    except wikipedia.exceptions.DisambiguationError as e:
        print(f"Disambiguation Error for '{page_title}': {e.options}")
        return None


def process_wikipedia_data(page_title):
    content = get_wikipedia_content(page_title)
    if content is None:
        return None, None  # Handle errors gracefully

    sentences = nltk.sent_tokenize(content)
    sentence_embeddings = model.encode(sentences)

    # Chunking (if needed for very large datasets)
    if len(sentences) > CHUNK_SIZE:
        all_embeddings = []
        for i in range(0, len(sentences), CHUNK_SIZE):
            chunk = sentences[i:i + CHUNK_SIZE]
            embeddings = model.encode(chunk)
            all_embeddings.extend(embeddings)  # Combine embeddings of chunks

        sentence_embeddings = np.array(all_embeddings)  # Convert to NumPy array

    kmeans = KMeans(n_clusters=NUM_CLUSTERS, random_state=0).fit(sentence_embeddings)
    labels = kmeans.labels_

    # Create FAISS index
    d = sentence_embeddings.shape[1]  # Dimension of embeddings
    index = faiss.IndexFlatL2(d)   # Use FlatL2 index for simplicity
    index.add(sentence_embeddings)

    return index, labels, sentences # return sentences along with other values


def search_similar_sentences(query, index, labels, sentences):
    query_embedding = model.encode([query])[0]  # Encode the query
    D, I = index.search(np.array([query_embedding]), k=5)  # Search for top 5 similar sentences
    # Get cluster label for the most similar result (refer to the most similar cluster)
    most_similar_cluster = labels[I[0][0]]

    similar_sentences = []

    # Retrieve sentences that belong to the most similar cluster
    for i in range(len(sentences)):
        if labels[i] == most_similar_cluster:
            similar_sentences.append(sentences[i])
    return similar_sentences

    # Alternatively retrieve all top k similar sentences regardless of the cluster
    # return [sentences[i] for i in I[0]]



if __name__ == "__main__":
    page_title = input("Enter a Wikipedia page title: ")
    index, labels, sentences = process_wikipedia_data(page_title)

    if index is not None and labels is not None and sentences is not None:  # Handle potential errors
        while True:
            query = input("Enter your search query (or type 'exit' to quit): ")
            if query.lower() == 'exit':
                break

            similar_sentences = search_similar_sentences(query, index, labels, sentences)

            if similar_sentences:
                print("\nSimilar sentences:")
                for sentence in similar_sentences:
                    print(sentence)
            else:
                print("No similar sentences found.")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Enter a Wikipedia page title: Amitabh Bachan 
Enter your search query (or type 'exit' to quit): Superhit films and awards

Similar sentences:
Bachchan is often hailed as the Shahenshah of Bollywood, Sadi Ke Mahanayak (translated as "Greatest actor of the century" in Hindi), Star of the Millennium, or simply Big B.
He is a recipient of several accolades including six National Film Awards and sixteen Filmfare Awards.
For Piku, he won his fourth National Film Award for Best Actor, making him the only actor to do so.
Bachchan has won numerous accolades in his career, including record four National Film Awards in Best Actor category and many awards at international film festivals and award ceremonies.
He has won sixteen Filmfare Awards and is the most nominated performer in any major acting category at Filmfare with 34 nominations in Best Actor and 42 nominations overall.
The Government of India honoured him with the Padma Shri in 1984, the Padma Bhushan in 2001, the Padma Vibhushan in 2015